In [1]:
#import statements
import numpy as np
import math
import re

#file name you'll save to
filename = 'SineBenchy.gcode'
#file name you'll read
readname = 'Benchy.gcode'

#GCODE sliced settings
    #THese are the params the input GCODE was sliced with
layerH = 0.2
width = 0.4

with open(filename, 'w') as file:
    file.write('; This code was generated by Leviticus Rhodens Non-planar code.\n')


These functions write to the output GCODE file in different ways.

In [2]:
#This cell is a collection of add to GCODE commands
def addPoint(x,y,z,e,f):
    '''This appends a new co-ordinate to the gcode file! It takes in x,y,z,and e co-ordinates as floats.'''
    
    with open(filename,'a') as file:
        if z<0:
            print("ERROR: I'm under the print bed!")
            return
        if e==0:
            file.write('G0 ')
        else:
            file.write('G1 ')
        file.write('F' + "{:#.8g}".format(f) + ' X' + "{:#.8g}".format(x) + ' Y' + "{:#.8g}".format(y) + ' Z' + "{:#.8g}".format(z) + ' E' + "{:#.8g}".format(e) + '\n')
        
def addComment(note):
    '''Takes in a string note and adds it as a comment to the GCODE file, does not need to have the ; in front'''
    with open(filename,'a') as file:
        file.write(";" + note +"\n")

def addString(string):
    '''Takes in a string and adds it to the output file, no questions asked and no notation added'''
    with open(filename,'a') as file:
        file.write(string)

These functions define the surfaces we want to interpolate between

In [3]:
#these functions define the surface maps we are transitioning between
#just make sure that the map at each point (0, z2 and z3) dosn't overlap with the other maps.
#ALso that Zhop in origonal code is higher than any non-linear function here

z2 = 100

def surface2(x,y):
    return 5*np.sin(x/2.5)*np.cos(y/5) + (y-110)/2
def surface3(x,y):
    
    return 0


def surface(x,y,z):
    '''This takes in the x,y, and planar z float, and returns the deltaZ from the planar z'''
    # README This is determined by YOU, and is able to change.
    
    if z<z2:
        dz = (z/z2)*surface2(x,y)
    else:
        dz = ((z-z2)/(z3-z2))*surface2(x,y)
    return dz

This function calculates the control volume of a pair of G1 commands

In [4]:
def calcControlVolume(X, Y, lastX, lastY, height, E):
    '''This Function takes in current and last co-ords, and calculates new, non-planar extrusion value.'''
    Z = height
    
    #This calculates volume of extrusion control volume and the planar volume, then finds the ratio of the two
    #This ratio is multiplied by the old extrusion value, preserving over/under extrusion in og GCODE
    #(GCODE takes a linear extrusion)
    dX = ((X-lastX)**2 + (Y-lastY)**2)**0.5
    dZ = surface(X,Y,Z) - surface(X,Y,Z-layerH)
    lastDZ = surface(lastX,lastY,Z) - surface(lastX,lastY,Z-layerH)
    vol = dX*width*(dZ + 0.5*(lastDZ-dZ) + layerH)
    oldVol = dX*width*layerH
    
    e = E * vol/oldVol
    
    return X, Y, Z+surface(X,Y,Z), e
            

These functions disect GCODE for floats

In [5]:
def extractCords(string):
    '''This function takes in a line of GCODE as a string and outputs co-ordinates in that string'''
    
    out = ['nan','nan','nan','nan','nan']
    coord = re.findall(r'[XYZEF].?\d+.\d+', string)
    
    for i in coord:
        if 'X' in i:
            out[0] = float(i[1:])
        if 'Y' in i:
            out[1] = float(i[1:])
        if 'Z' in i:
            out[2] = float(i[1:])
        if 'E' in i:
            out[3] = float(i[1:])
        if 'F' in i:
            out[4] = float(i[1:])
            
    return out
            
    

This guy puts it all together!

In [6]:
#This loop iterates over every line of text in the input GCODE, and modifies the code so movement is non planar.
#it also corrects extrusion rate

layer = 1
height = 0
lastCoords = [0,0,0,0,1200]

with open(readname) as file:
    for line in file:
        #This level is called for every line of input GCODE
        if 'G0' in line:
            coords = extractCords(line)
            
            if coords[0] == 'nan':
                coords[0] = lastCoords[0]
            if coords[1] == 'nan':
                coords[1] = lastCoords[1]
            if coords[2] == 'nan':
                coords[2] = lastCoords[2]
            if coords[3] == 'nan':
                coords[3] = lastCoords[3]
            
            #Here we check if our path is longer than a threshold, so we dont crash into the base
            dist = ((coords[0]-lastCoords[0])**2 + (coords[1]-lastCoords[1])**2 + (coords[2]-lastCoords[2])**2)**0.5
            if dist >= 0.5:
                iterations = int(dist/0.5)
            else:
                iterations = 1
                
            lastX = lastCoords[0]
            lastY = lastCoords[1]
            
            for i in list(range(1, iterations+1)):
                t = i/iterations
                x = t*coords[0] + (1-t)*lastCoords[0]
                y = t*coords[1] + (1-t)*lastCoords[1]
                #Literally just call this for the zPt
                xPt, yPt, zPt, e = calcControlVolume(x,y,lastX,lastY,coords[2], 10)

                addPoint(x,y,zPt,0,1200)
                
                lastX = x
                lastY = y

            for i in [0,1,2,3,4]:
                if 'nan' != coords[i]:
                    lastCoords[i] = coords[i]
            
        elif 'G1' in line:
            coords = extractCords(line)
            
            if coords[0] == 'nan':
                coords[0] = lastCoords[0]
            if coords[1] == 'nan':
                coords[1] = lastCoords[1]
            if coords[2] == 'nan':
                coords[2] = lastCoords[2]
            if coords[3] == 'nan':
                coords[3] = lastCoords[2]
                
            dist = ((coords[0]-lastCoords[0])**2 + (coords[1]-lastCoords[1])**2 + (coords[2]-lastCoords[2])**2)**0.5
            if dist >= 0.5:
                iterations = int(dist/0.5)
            else:
                iterations = 1
            
            lastX = lastCoords[0]
            lastY = lastCoords[1]
            
            for i in list(range(1, iterations+1)):
                
                t = i/iterations
                x = t*coords[0] + (1-t)*lastCoords[0]
                y = t*coords[1] + (1-t)*lastCoords[1]
                
                xPt, yPt, zPt, e = calcControlVolume(x,y,lastX,lastY,coords[2], coords[3])

                addPoint(xPt, yPt, zPt, e, 1200)

                lastX = x
                lastY = y
                
            for i in [0,1,2,3,4]:
                    if 'nan' != coords[i]:
                        lastCoords[i] = coords[i]
        
        elif 'LAYER:' in line:
            layer += 1
            height = (layer-1)*layerH
            addString(line)
            
        else:
            addString(line)
            
        

/tmp/ipykernel_1371/596732003.py:14: RuntimeWarning: invalid value encountered in double_scalars
  e = E * vol/oldVol
